In [6]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')


data = pd.read_csv('dataset/전국스마트가로등표준데이터-20210801.xls',encoding='cp949')

data.isnull().sum()

data.columns

data['시도명'].unique()

data['시도명'] = data['시도명'].replace('14','전라남도')

data[data['시도명']=='전라남도']['시군구명'].unique()

jl = data[data['시도명']=='전라남도']
sinan_1 = jl[jl['시군구명']=='신안군']

import re
def dong(x):
    dong = re.sub('전라남도 신안군',"",x)
    dong = dong.lstrip()
    comp = re.compile('[가-힣]+')
    dong = comp.match(dong)
    dong = dong.group()
    return dong

sinan_1['읍'] = sinan_1['소재지지번주소'].apply(lambda x : dong(x))

sinan_1['설치유무'] = 1

get_col = ['시도명','시군구명','소재지지번주소','읍','설치유무']
sinan_1 = sinan_1[get_col]

u1 =['지도읍' ,'압해읍' ,'증도면' ,'임자면' ,'자은면' ,'비금면' ,'도초면' ,'흑산면' ,'하의면' ,'신의면' ,'장산면' ,'안좌면' ,'팔금면','암태면']
u2 = sinan_1['읍'].unique().tolist()
us = pd.Series(u1)

sinan = pd.DataFrame(us)
sinan['설치유무'] = np.nan
sinan['시도명'] = data[data['시도명']=='전라남도']['시도명'].unique()[0]
sinan['시군구명'] = data[data['시군구명']=='신안군']['시군구명'].unique()[0]

def 설치유무(x):
    y = ''
    for i in u2 :
        if x == i :
            y = 1
    return y
sinan['설치유무'].update(sinan[0].apply(설치유무))
sinan['설치유무'] = sinan['설치유무'].replace("",0)
sinan = sinan.rename(columns={0:'읍/면/동'})

sinan = sinan[['시도명','시군구명','읍/면/동','설치유무']]

sinan

UnicodeDecodeError: 'cp949' codec can't decode byte 0xb1 in position 5: illegal multibyte sequence

In [ ]:
fire = pd.read_csv('dataset/화재발생정보.csv', encoding='cp949',index_col='연번')
fire = fire[(fire['시도']=='전라남도') & (fire['시군구'] == '신안군')]
fire['읍면동'].value_counts()
group = fire.groupby('읍면동').count()
group = group.reset_index(drop = False)
group['화재발생횟수'] = group['사망']
fire = group[['읍면동','화재발생횟수']]
fire = fire_s.rename(columns={'읍면동':'읍/면/동'})
fire['화재발생비율'] = fire['화재발생횟수']/fire['화재발생횟수'].sum()
fire['화재발생비율'] = fire['화재발생비율'].round(2)
df = pd.merge(sinan,fire,on='읍/면/동',how='outer')
df